## The Builder Pattern
The main difference is that a Factory pattern creates an object in a single step, whereas a Builder pattern creates an object in multiple steps, and almost always through the use of a director. A new computer analogy:

In [2]:
MINI14 = '1.4GHz Mac mini'

class AppleFactory:
    ''' here we use a Factory pattern to create a computer 
        the nested MacMini14 class is a neat way of forbidding direct class instantiation
    '''
    class MacMini14:
        def __init__(self):
            self.memory = 4 # in gigabytes
            self.hdd = 500  # in gigabytes
            self.gpu = 'Intel HD Graphics 5000'
            
        def __str__(self):
            info = ('Model: {}'.format(MINI14),
            'Memory: {}GB'.format(self.memory),
            'Hard Disk: {}GB'.format(self.hdd),
            'Graphics Card: {}'.format(self.gpu))
            return '\n'.join(info)

    def build_computer(self, model):
        if (model == MINI14):
            return self.MacMini14()
        else:
            print("I don't know how to build {}".format(model))

if __name__ == '__main__':
    afac = AppleFactory()
    mac_mini = afac.build_computer(MINI14)
    print(mac_mini)

Model: 1.4GHz Mac mini
Memory: 4GB
Hard Disk: 500GB
Graphics Card: Intel HD Graphics 5000


Another option is to build a custom PC rather than a specific preconfigured computer model. In this case, you use the Builder pattern.

In [9]:
class Computer:
    def __init__(self, serial_number):
        self.serial = serial_number
        self.memory = None # in gigabytes
        self.hdd = None # in gigabytes
        self.gpu = None
        
    def __str__(self):
        info = ('Memory: {}GB'.format(self.memory),
        'Hard Disk: {}GB'.format(self.hdd),
        'Graphics Card: {}'.format(self.gpu))
        return '\n'.join(info)


In [6]:
class ComputerBuilder:
    def __init__(self):
        self.computer = Computer('AG23385193')
        
    def configure_memory(self, amount):
        self.computer.memory = amount
        
    def configure_hdd(self, amount):
        self.computer.hdd = amount
        
    def configure_gpu(self, gpu_model):
        self.computer.gpu = gpu_model


In [7]:
class HardwareEngineer:
    def __init__(self):
        self.builder = None
        
    def construct_computer(self, memory, hdd, gpu):
        self.builder = ComputerBuilder()
        [step for step in (self.builder.configure_memory(memory),
        self.builder.configure_hdd(hdd),
        self.builder.configure_gpu(gpu))]
        
    @property
    def computer(self):
        return self.builder.computer


In [10]:
def main():
    engineer = HardwareEngineer()
    engineer.construct_computer(hdd=500, memory=8, gpu='GeForce GTX 650 Ti')
    computer = engineer.computer
    print(computer)
                            
if __name__ == '__main__':
    main()

Memory: 8GB
Hard Disk: 500GB
Graphics Card: GeForce GTX 650 Ti


## Implementation example
The Builder design pattern for a pizza ordering application.

In [11]:
from enum import Enum
import time

PizzaProgress = Enum('PizzaProgress', 'queued preparation baking ready')
PizzaDough = Enum('PizzaDough', 'thin thick')
PizzaSauce = Enum('PizzaSauce', 'tomato creme_fraiche')
PizzaTopping = Enum('PizzaTopping', 'mozzarella double_mozzarella bacon ham mushrooms red_onion oregano')
STEP_DELAY = 3 # in seconds for the sake of the example


In [12]:
class Pizza:
    def __init__(self, name):
        self.name = name
        self.dough = None
        self.sauce = None
        self.topping = []
        
    def __str__(self):
        return self.name
    
    def prepare_dough(self, dough):
        self.dough = dough
        print('preparing the {} dough of your {}...'.format(self.dough.name, self))
        time.sleep(STEP_DELAY)
        print('done with the {} dough'.format(self.dough.name))


In [13]:
class MargaritaBuilder:
    def __init__(self):
        self.pizza = Pizza('margarita')
        self.progress = PizzaProgress.queued
        self.baking_time = 5 # in seconds for the sake of the example
        
    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thin)
        
    def add_sauce(self):
        print('adding the tomato sauce to your margarita...')
        self.pizza.sauce = PizzaSauce.tomato
        time.sleep(STEP_DELAY)
        print('done with the tomato sauce')
        
    def add_topping(self):
        print('adding the topping (double mozzarella, oregano) to your margarita')
        self.pizza.topping.append([i for i in (PizzaTopping.double_mozzarella, PizzaTopping.oregano)])
        time.sleep(STEP_DELAY)
        print('done with the topping (double mozzarrella, oregano)')
          
    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your margarita for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your margarita is ready')


In [18]:
class CreamyBaconBuilder:
    def __init__(self):
        self.pizza = Pizza('creamy bacon')
        self.progress = PizzaProgress.queued
        self.baking_time = 7 # in seconds for the sake of the example
        
    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thick)
        
    def add_sauce(self):
        print('adding the crème fraîche sauce to your creamy bacon')
        self.pizza.sauce = PizzaSauce.creme_fraiche
        time.sleep(STEP_DELAY)
        print('done with the crème fraîche sauce')
        
    def add_topping(self):
        print('adding the topping (mozzarella, bacon, ham, mushrooms, red onion, oregano) to your creamy bacon')
        self.pizza.topping.append([t for t in
        (PizzaTopping.mozzarella, PizzaTopping.bacon,
        PizzaTopping.ham,PizzaTopping.mushrooms,
        PizzaTopping.red_onion, PizzaTopping.oregano)])
        time.sleep(STEP_DELAY)
        print('done with the topping (mozzarella, bacon, ham, mushrooms, red onion, oregano)')
        
    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your creamy bacon for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your creamy bacon is ready')


In [14]:
class Waiter:
    def __init__(self):
        self.builder = None
        
    def construct_pizza(self, builder):
        self.builder = builder
        [step() for step in (builder.prepare_dough, builder.add_sauce, builder.add_topping, builder.bake)]
        
    @property
    def pizza(self):
        return self.builder.pizza


In [15]:
def validate_style(builders):
    try:
        pizza_style = raw_input('What pizza would you like, [m]argarita or [c]reamy bacon? ')
        builder = builders[pizza_style]()
        valid_input = True
    except KeyError as err:
        print('Sorry, only margarita (key m) and creamy bacon (key c) are available')
        return (False, None)
    return (True, builder)


In [20]:
def main():
    builders = dict(m=MargaritaBuilder, c=CreamyBaconBuilder)
    valid_input = False
    while not valid_input:
        valid_input, builder = validate_style(builders)
    print("")
    waiter = Waiter()
    waiter.construct_pizza(builder)
    pizza = waiter.pizza
    print("")
    print('Enjoy your {}!'.format(pizza))

if __name__ == '__main__':
    main()

What pizza would you like, [m]argarita or [c]reamy bacon? m

preparing the thin dough of your margarita...
done with the thin dough
adding the tomato sauce to your margarita...
done with the tomato sauce
adding the topping (double mozzarella, oregano) to your margarita
done with the topping (double mozzarrella, oregano)
baking your margarita for 5 seconds
your margarita is ready

Enjoy your margarita!


## A Fluent builder pattern
In this variation of the builder pattern calls to the builder methods are chained. This is accomplished by defining the builder itself as an inner class and returning itself from each of the setter-like methods on it. The build() method returns the final object.

In [21]:
class Pizza:
    def __init__(self, builder):
        self.garlic = builder.garlic
        self.extra_cheese = builder.extra_cheese
        
    def __str__(self):
        garlic = 'yes' if self.garlic else 'no'
        cheese = 'yes' if self.extra_cheese else 'no'
        info = ('Garlic: {}'.format(garlic), 'Extra cheese: {}'.format(cheese))
        return '\n'.join(info)
    
    class PizzaBuilder:
        def __init__(self):
            self.extra_cheese = False
            self.garlic = False
            
        def add_garlic(self):
            self.garlic = True
            return self
        
        def add_extra_cheese(self):
            self.extra_cheese = True
            return self
        
        def build(self):
            return Pizza(self)
        
if __name__ == '__main__':
    pizza = Pizza.PizzaBuilder().add_garlic().add_extra_cheese().build()
    print(pizza)


Garlic: yes
Extra cheese: yes


# A Row Builder

In [26]:
FISCAL_YEAR = 2
BATCH_ID = 3

class Row_Builder(object):
    
    def __init__(self):
        self.row = ['' for i in range(10)]

    def with_fy(self, fiscal_year):
        self.row[FISCAL_YEAR] = fiscal_year
        return self

    def with_id(self, batch_id):
        self.row[BATCH_ID] = batch_id
        return self

    def build(self):
        return self.row
    
row_FY13_888 = Row_Builder().with_fy('FY13').with_id('888').build()
row_FY13_888

['', '', 'FY13', '888', '', '', '', '', '', '']

# A car builder
From https://gist.github.com/pazdera/1121157

In [27]:
class Director:
    
    """ Controls the construction process.
        Director has a builder associated with him. Director then delegates building of the smaller parts
        to the builder and assembles them together.
    """

    _builder = None

    def setBuilder(self, builder):
        self._builder = builder

    # The algorithm for assembling a car
    def getCar(self):
        car = Car()

        # First goes the body
        body = self._builder.getBody()
        car.setBody(body)

        # Then engine
        engine = self._builder.getEngine()
        car.setEngine(engine)

        # And four wheels
        [car.attachWheel(self._builder.getWheel()) for i in range(4)]

        return car


In [28]:
class Car:
    
    """ The final product.
        A car is assembled by the 'Director' class from
        parts made by 'Builder'. Both these classes have influence on the resulting object.
    """

    def __init__(self):
        self._wheels  = list()
        self._engine  = None
        self._body    = None

    def setBody(self, body):
        self._body = body

    def attachWheel(self, wheel):
        self._wheels.append(wheel)

    def setEngine(self, engine):
        self._engine = engine

    def specification(self):
        print "body: %s" % self._body.shape
        print "engine horsepower: %d" % self._engine.horsepower
        print "tire size: %d\'" % self._wheels[0].size


In [31]:
class Builder:

    """ Creates various parts of a vehicle.
        This class is responsible for constructing all
        the parts for a vehicle.
    """

    def getWheel(self): pass
    def getEngine(self): pass
    def getBody(self): pass


In [32]:
class JeepBuilder(Builder):

    """ Concrete Builder implementation.
        This class builds parts for Jeep's SUVs.
    """

    def getWheel(self):
        wheel = Wheel()
        wheel.size = 22
        return wheel

    def getEngine(self):
        engine = Engine()
        engine.horsepower = 400
        return engine

    def getBody(self):
        body = Body()
        body.shape = "SUV"
        return body


In [33]:
class NissanBuilder(Builder):

    """ Concrete Builder implementation.
        This class builds parts for Nissan's family cars.
    """

    def getWheel(self):
        wheel = Wheel()
        wheel.size = 16
        return wheel

    def getEngine(self):
        engine = Engine()
        engine.horsepower = 85
        return engine

    def getBody(self):
        body = Body()
        body.shape = "hatchback"
        return body


In [35]:
# Car parts
class Wheel:
    size = None

class Engine:
    horsepower = None

class Body:
    shape = None


In [36]:
def main():
    jeepBuilder = JeepBuilder()
    nissanBuilder = NissanBuilder()
    director = Director()

    # Build Jeep
    print "Jeep"
    director.setBuilder(jeepBuilder)
    jeep = director.getCar()
    jeep.specification()

    print ""

    # Build Nissan
    print "Nissan"
    director.setBuilder(nissanBuilder)
    nissan = director.getCar()
    nissan.specification()

if __name__ == "__main__":
    main()

Jeep
body: SUV
engine horsepower: 400
tire size: 22'

Nissan
body: hatchback
engine horsepower: 85
tire size: 16'


## From Source Making: Builder in Python
https://sourcemaking.com/design_patterns/builder/python/1

In [40]:
"""
Separate the construction of a complex object from its representation so
that the same construction process can create different representations.
"""
import six
import abc


class Director:
    """
    Construct an object using the Builder interface.
    """

    def __init__(self):
        self._builder = None

    def construct(self, builder):
        self._builder = builder
        self._builder._build_part_a()
        self._builder._build_part_b()
        self._builder._build_part_c()
        
    @property
    def product(self):
        if self._builder is not None:
            return self._builder.product
    

@six.add_metaclass(abc.ABCMeta)
class Builder():
    """
    Specify an abstract interface for creating parts of a Product
    object.
    """

    def __init__(self):
        self.product = Product()

    @abc.abstractmethod
    def _build_part_a(self):
        pass

    @abc.abstractmethod
    def _build_part_b(self):
        pass

    @abc.abstractmethod
    def _build_part_c(self):
        pass


class ConcreteBuilder(Builder):
    """
    Construct and assemble parts of the product by implementing the
    Builder interface.
    Define and keep track of the representation it creates.
    Provide an interface for retrieving the product.
    """

    def _build_part_a(self):
        pass

    def _build_part_b(self):
        pass

    def _build_part_c(self):
        pass


class Product:
    """
    Represent the complex object under construction.
    """

    pass


def main():
    concrete_builder = ConcreteBuilder()
    director = Director()
    director.construct(concrete_builder)
    print(director.product)

if __name__ == "__main__":
    main()

<__main__.Product instance at 0x0000000003F5FD08>
